## Students info:
1. Name: Donia Gameel Mahmoud

   ID: 9202523

2. Name: Heba Ashraf Raslan

   ID: 9203667

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.chdir("/content/drive/My Drive")

In [5]:
import zipfile

# Specify the name of your .zip file
zip_file_name = "in.zip"

# Specify the directory where you want to extract the contents
extract_to_directory = "/content/drive/My Drive"

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=df63856b0b232d20584707b2308f117bdd2808f2dd9d3b18881a0231191dfbcf
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [9]:
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [10]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("prepaidCount")\
    .getOrCreate()

In [11]:
sc=spark.sparkContext

In [13]:
input_csv="in.csv"

In [17]:
vodafone_rdd=sc.textFile(input_csv)

In [18]:
vodafone_rdd.take(100) # line by line

['224,10',
 '836,100',
 '81,5',
 '809,50',
 '786,25',
 '63,5',
 '478,25',
 '330,10',
 '511,15',
 '450,25',
 '521,25',
 '150,50',
 '671,5',
 '971,5',
 '439,10',
 '536,50',
 '253,5',
 '425,15',
 '638,50',
 '841,10',
 '324,10',
 '125,50',
 '937,15',
 '898,5',
 '461,50',
 '295,10',
 '683,10',
 '331,25',
 '187,15',
 '873,100',
 '303,50',
 '119,5',
 '589,100',
 '742,15',
 '81,25',
 '47,100',
 '521,50',
 '617,15',
 '228,5',
 '891,15',
 '107,10',
 '59,25',
 '526,100',
 '461,15',
 '426,100',
 '389,25',
 '114,5',
 '866,5',
 '592,50',
 '452,100',
 '210,5',
 '607,15',
 '628,100',
 '500,25',
 '815,25',
 '358,25',
 '832,50',
 '667,5',
 '708,5',
 '568,5',
 '536,15',
 '775,100',
 '280,25',
 '227,5',
 '461,100',
 '462,25',
 '3,100',
 '410,10',
 '67,5',
 '872,5',
 '589,25',
 '488,50',
 '601,50',
 '393,5',
 '148,15',
 '75,50',
 '742,10',
 '177,25',
 '875,15',
 '121,50',
 '38,50',
 '445,100',
 '87,50',
 '730,15',
 '552,10',
 '790,10',
 '653,100',
 '586,15',
 '400,100',
 '994,15',
 '110,100',
 '84,25',
 '9

In [19]:
vodafone_rdd.count() ## number of lines

75000000

In [26]:
vodafone_mapper = vodafone_rdd.map(lambda line: (int(line.split(',')[0]), int(line.split(',')[1])))

In [27]:
vodafone_mapper.take(5)

[(224, 10), (836, 100), (81, 5), (809, 50), (786, 25)]

In [28]:
vodafone_reducer=vodafone_mapper.reduceByKey(lambda x,y:(x+y)).sortByKey()

In [29]:
vodafone_reducer.take(40)

[(0, 2554710),
 (1, 2540660),
 (2, 2562385),
 (3, 2545740),
 (4, 2543400),
 (5, 2569725),
 (6, 2559895),
 (7, 2563020),
 (8, 2580355),
 (9, 2565660),
 (10, 2525035),
 (11, 2561655),
 (12, 2567315),
 (13, 2548960),
 (14, 2561640),
 (15, 2544125),
 (16, 2566590),
 (17, 2561645),
 (18, 2573245),
 (19, 2564715),
 (20, 2561125),
 (21, 2597905),
 (22, 2546515),
 (23, 2566060),
 (24, 2580670),
 (25, 2553140),
 (26, 2551620),
 (27, 2577430),
 (28, 2560910),
 (29, 2564450),
 (30, 2572195),
 (31, 2572830),
 (32, 2566775),
 (33, 2551710),
 (34, 2561460),
 (35, 2572100),
 (36, 2541460),
 (37, 2560805),
 (38, 2566170),
 (39, 2527425)]

In [34]:
output_directory = "/content/outputDirectory"

# Combine all partitions into a single partition
vodafone_reducer_single_partition = vodafone_reducer.coalesce(1)

# Save the reducer result to a single text file named "out.txt"
vodafone_reducer_single_partition.saveAsTextFile(output_directory)

# Rename the part-00000 file to out.txt
import os
os.rename(output_directory + "/part-00000", output_directory + "/out.txt")